<a href="https://colab.research.google.com/github/yucachen/MLTSA25_YChen/blob/main/In_Class_Code/Copy_of_TimeSeriesClassificationWithKerasTransformers_ClassCopy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import keras
from keras import layers

Attaching encoder that uses more attention-based

Output of encoder and then feed it into the model

In [8]:
def readucr(filename):
    data = np.loadtxt(filename, delimiter="\t")
    y = data[:, 0]
    x = data[:, 1:]
    return x, y.astype(int)


root_url = "https://raw.githubusercontent.com/hfawaz/cd-diagram/master/FordA/"

x_train, y_train = readucr(root_url + "FordA_TRAIN.tsv")
x_test, y_test = readucr(root_url + "FordA_TEST.tsv")

x_train = x_train.reshape((x_train.shape[0], x_train.shape[1], 1))
x_test = x_test.reshape((x_test.shape[0], x_test.shape[1], 1))

n_classes = len(np.unique(y_train))

idx = np.random.permutation(len(x_train))
x_train = x_train[idx]
y_train = y_train[idx]

y_train[y_train == -1] = 0
y_test[y_test == -1] = 0

input_shape = x_train.shape[1:]

In [9]:
def train_and_evaluate(model, Xtrn, Xtst, Ytrn, Ytst):
    cb_early = keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True) # callback: if isn't improving after 10 epochs of training, will restore the weights of the model that gave the best results (prevents overfitting and saves computational resources)
    model.fit(
        Xtrn,
        Ytrn,
        epochs=100,
        batch_size=32,
        validation_split=0.2,
        callbacks=[cb_early],
        verbose=0,
    )
    loss_tst, acc_tst = ...
    print(f"Loss calculated on the testing set: {loss_tst:.4f}")
    print(f"Accuracy calculated on the testing set: {acc_tst:.4f}")
    return None

In [12]:
def transformer_block(
    x,
    input_shape,
    heads=4,
    key_dim=256,
    encoder_proj_dim=4,
    dropout_attention=0.25,
    dropout_projection=0.25,
):
    x0 = layers.MultiHeadAttention(num_heads=heads, key_dim=key_dim)(x, x) # parallel instances
    x0 = layers.Dropout(dropout_attention)(x0)
    x0 = layers.Add()([x, x0]) # the "residual connection"
    x0 = layers.LayerNormalization()(x0)


    # 1D convolutions
    x1 = layers.Conv1D(filters=encoder_proj_dim, kernel_size=1, activation="relu")(x0) # projecting to higher dimension and then back down again
    x1 = layers.Conv1D(filters=input_shape[1], kernel_size=1,  activation="relu")(x1)
    x1 = layers.Dropout(dropout_projection)(x1)
    x1 = layers.Add()([x0, x1])
    x1 = layers.LayerNormalization()(x1)

    return x1


def build_transformer_model(
    input_shape,
    encoder_blocks=4,
    feed_forward_units=[28],
):
    inputs = keras.Input(shape=input_shape)
    x = inputs

    for _ in range(encoder_blocks):
        x = transformer_block(x, input_shape)

    x = layers.GlobalMaxPooling1D(data_format="channels_first")(x) # reduce dimensionality
    for n in feed_forward_units:
        x = layers.Dense(n, activation="relu")(x)
        x = layers.Dropout(0.4)(x)

    outputs = layers.Dense(2, activation="softmax")(x)
    model = keras.Model(inputs, outputs)

    return model


transformer_model = build_transformer_model(input_shape)
transformer_model.compile(
    loss="sparse_categorical_crossentropy",
    optimizer=keras.optimizers.Adam(learning_rate=1e-4),
    metrics=["sparse_categorical_accuracy"],
)
transformer_model.summary()

train_and_evaluate(transformer_model, x_train, x_test, y_train, y_test)

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_4       │ (None, 500, 1)    │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multi_head_attenti… │ (None, 500, 1)    │      7,169 │ input_layer_4[0]… │
│ (MultiHeadAttentio… │                   │            │ input_layer_4[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_4 (Dropout) │ (None, 500, 1)    │          0 │ multi_head_atten… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_2 (Add)         │ (None, 500, 1)    │          0 │ input_layer_4[0]… │
│                     │                   │            │ dropout_4[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalizatio… │ (None, 500, 1)    │          2 │ add_2[0][0]       │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_2 (Conv1D)   │ (None, 500, 4)    │          8 │ layer_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_3 (Conv1D)   │ (None, 500, 1)    │          5 │ conv1d_2[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_5 (Dropout) │ (None, 500, 1)    │          0 │ conv1d_3[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_3 (Add)         │ (None, 500, 1)    │          0 │ layer_normalizat… │
│                     │                   │            │ dropout_5[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalizatio… │ (None, 500, 1)    │          2 │ add_3[0][0]       │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multi_head_attenti… │ (None, 500, 1)    │      7,169 │ layer_normalizat… │
│ (MultiHeadAttentio… │                   │            │ layer_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_7 (Dropout) │ (None, 500, 1)    │          0 │ multi_head_atten… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_4 (Add)         │ (None, 500, 1)    │          0 │ layer_normalizat… │
│                     │                   │            │ dropout_7[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalizatio… │ (None, 500, 1)    │          2 │ add_4[0][0]       │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_4 (Conv1D)   │ (None, 500, 4)    │          8 │ layer_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_5 (Conv1D)   │ (None, 500, 1)    │          5 │ conv1d_4[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_8 (Dropout) │ (None, 500, 1)    │          0 │ conv1d_5[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_5 (Add)         │ (None, 500, 1)    │          0 │ layer_normalizat… │
│                     │                   │            │ dropout_8[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalizatio… │ (None, 500, 1)    │          2 │ add_5[0][0]       │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼─────────────────

 Total params: 42,830 (167.30 KB)

 Trainable params: 42,830 (167.30 KB)

 Non-trainable params: 0 (0.00 B)

TypeError: cannot unpack non-iterable ellipsis object

In [ ]:
def build_ff_model(
    input_shape,
    feed_forward_units=[128]
):
    inputs = ...
    x = inputs

    x = ...
    for ... in ...:
        x = ...
        x = ...

    outputs = ...
    model = keras.Model(..., ...)

    return model


input_shape = x_train.shape[1:]
ff_model = build_ff_model(input_shape)
ff_model.compile(
    loss="sparse_categorical_crossentropy",
    optimizer=keras.optimizers.Adam(learning_rate=...),
    metrics=["sparse_categorical_accuracy"],
)
ff_model.summary()

train_and_evaluate(ff_model, x_train, x_test, y_train, y_test)